<a href="https://colab.research.google.com/github/SpringBoardMentor193s/SentinelNet_Oct_Batch/blob/Aryan_Sahu/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
column_names = [
'duration', 'protocol_type', 'service', 'flag',
'src_bytes', 'dst_bytes',
'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins',
'logged_in', 'num_compromised',
'root_shell', 'su_attempted',
'num_root', 'num_file_creations',
'num_shells', 'num_access_files',
'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count',
'srv_count', 'serror_rate', 'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate',
'dst_host_count', 'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'class', 'difficulty' ]

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Load data
train_df = pd.read_csv('/content/drive/MyDrive/nsl-kdd/KDDTrain+.txt', header=None, names=column_names)
train_df.drop(['difficulty'], axis=1, inplace=True)

test_df = pd.read_csv('/content/drive/MyDrive/nsl-kdd/KDDTest+.txt', header=None, names=column_names)
test_df.drop(['difficulty'], axis=1, inplace=True)

# Display data
print("--- Training Data ---")
print(f"Shape: {train_df.shape}")
print(train_df.head())

print("--- Testing Data ---")
print(f"Shape: {test_df.shape}")
print(test_df.head())


--- Training Data ---
Shape: (125973, 42)
   duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0         0           tcp  ftp_data   SF        491          0     0   
1         0           udp     other   SF        146          0     0   
2         0           tcp   private   S0          0          0     0   
3         0           tcp      http   SF        232       8153     0   
4         0           tcp      http   SF        199        420     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                  25   
1               0       0    0  ...                   1   
2               0       0    0  ...                  26   
3               0       0    0  ...                 255   
4               0       0    0  ...                 255   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.17                    0.03   
1                    0.00                    0.60   
2          

In [20]:
    df.head(5)

NameError: name 'df' is not defined